In [13]:
from utils import prepare_training_data
path = 'D:\database\wiketext\sub_new\ch_hdf5'
device = 'cpu' 
# trainloader = prepare_training_data(256,num_jobs=7,path=path,type_='test',device=device,train_batch_size=32)

In [2]:
30528/8

3816.0

## model output

In [14]:
import torch
# from typing import Final, Any, Callable
import modeling

In [15]:
class BertPretrainingCriterion(torch.nn.Module):

    def __init__(self, vocab_size, sequence_output_is_dense=False):
        super(BertPretrainingCriterion, self).__init__()
        self.loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)
        self.vocab_size = vocab_size
        self.sequence_output_is_dense = sequence_output_is_dense

    def forward(self, prediction_scores, seq_relationship_score, masked_lm_labels, next_sentence_labels):
        if self.sequence_output_is_dense:
            # prediction_scores are already dense
            masked_lm_labels_flat = masked_lm_labels.view(-1)
            mlm_labels = masked_lm_labels_flat[masked_lm_labels_flat != -1]
            masked_lm_loss = self.loss_fn(prediction_scores.view(-1, self.vocab_size), mlm_labels.view(-1))
        else:
            masked_lm_loss = self.loss_fn(prediction_scores.view(-1, self.vocab_size), masked_lm_labels.view(-1))
        next_sentence_loss = self.loss_fn(seq_relationship_score.view(-1, 2), next_sentence_labels.view(-1))
        total_loss = masked_lm_loss + next_sentence_loss
        return total_loss,masked_lm_loss,next_sentence_loss

In [4]:
import h5py
path1 = 'D:\database\wiketext\sub_new\enhdf5\\test_0.hdf5'
file = h5py.File(path1, 'r')

In [6]:
file.keys()

<KeysViewHDF5 ['input_ids', 'input_mask', 'masked_lm_ids', 'masked_lm_positions', 'next_sentence_labels', 'segment_ids']>

In [17]:
config = modeling.BertConfig.from_json_file('base.json')
model = modeling.BertForPreTraining(config, sequence_output_is_dense=False)
criterion = BertPretrainingCriterion(config.vocab_size, sequence_output_is_dense=False)
# model.to(device='cuda')
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta', 'LayerNorm']

optimizer_grouped_parameters = [
{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
{'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

optimizer = torch.optim.Adam(params=optimizer_grouped_parameters,lr=5e-5)

In [18]:
optimizer.state_dict()

{'state': {},
 'param_groups': [{'weight_decay': 0.01,
   'lr': 5e-05,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'amsgrad': False,
   'maximize': False,
   'params': [0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77]},
  {'weight_decay': 0.0,
   'lr': 5e-05,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'amsgrad': False,
   'maximize': False,
   'params': [78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
 

In [ ]:
#  ['input_ids', 'input_mask', 'masked_lm_ids', 
# # 'masked_lm_positions', 'next_sentence_labels', 'segment_ids']>
# all_input_ids 
# all_input_mask 
# all_segment_ids
# all_label_ids 
# next_sentence_labels 

In [6]:
trainloader = prepare_training_data(20,num_jobs=1,path=path,type_='test',device=device,train_batch_size=16,
dense_output=True,)

100%|██████████| 1/1 [00:00<00:00, 21.81it/s]

In [7]:
for batch in trainloader:
    for i in batch:
        print(i.shape)
    break

torch.Size([16, 256])
torch.Size([16, 256])
torch.Size([16, 256])
torch.Size([16, 256])
torch.Size([16])


In [8]:
batch[3].view(-1).shape

torch.Size([4096])

In [8]:
prediction_scores.shape

torch.Size([16, 256, 30528])

In [9]:
device = 'cuda'
torch.cuda.empty_cache()
for batch in trainloader:
    all_input_ids  = batch[0].to(device)
    all_input_mask = batch[1].to(device) 
    all_segment_ids= batch[2].to(device)
    all_label_ids  = batch[3].to(device)
    next_sentence_labels = batch[4].to(device)
    prediction_scores, seq_relationship_score = model(input_ids=all_input_ids, token_type_ids=all_label_ids,
             attention_mask=all_input_mask, masked_lm_labels=all_label_ids)
    total_loss,masked_lm_loss,next_sentence_loss = criterion(prediction_scores, seq_relationship_score, all_label_ids,
             next_sentence_labels)
    total_loss.backward()
    optimizer.step()
    print(f'prediction_scores"{prediction_scores.shape}, seq_relationship_score : {seq_relationship_score.shape}')
    print(f'all_label_ids"{all_label_ids.shape}, next_sentence_labels : {next_sentence_labels.shape}')
    print(f"total loss:{total_loss.item()} , masked_loss :{masked_lm_loss.item()}, next_sentence_loss: {next_sentence_loss.item()}")
    break

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [8]:
next_sentence_loss

tensor(0.7014, device='cuda:0', grad_fn=<NllLossBackward0>)

In [18]:
from torch.nn import CrossEntropyLoss
loss = CrossEntropyLoss(ignore_index=-1)
loss(prediction_scores.view(-1, 30528),batch[2].view(-1))


tensor(10.6123, device='cuda:0', grad_fn=<NllLossBackward0>)

In [1]:
bert_preprocessing_command = 'python my_pretraining.py'
bert_preprocessing_command += ' --input_dir=' + 'D:\database\wiketext\sub_new\en_30528_hdf5'
bert_preprocessing_command += ' --config_file=' + 'base.json'
bert_preprocessing_command += ' --output_dir=' + 'output'
bert_preprocessing_command += ' --train_batch_size='+str(12)
bert_preprocessing_command += ' --learning_rate=' + str(5e-5)
bert_preprocessing_command += ' --num_train_epochs=' + str(10)
bert_preprocessing_command += ' --max_steps=' + str(1000)
bert_preprocessing_command += ' --warmup_proportion=' + str(0.01)

bert_preprocessing_command += ' --num_workers=' + str(4)
print(bert_preprocessing_command)

python my_pretraining.py --input_dir=D:\database\wiketext\sub_new\en_30528_hdf5 --config_file=base.json --output_dir=output --train_batch_size=32 --learning_rate=5e-05 --num_train_epochs=10 --max_steps=1000 --warmup_proportion=0.01 --num_workers=4


In [1]:
en_30528_hdf5 = 'D:\database\wiketext\sub_new\en_30528_hdf5'
from utils import prepare_training_data
trainloader = prepare_training_data(5,num_jobs=1,path=en_30528_hdf5,type_='training',
                                    train_batch_size=32,dense_output=False,device='cpu')

100%|██████████| 5/5 [00:03<00:00,  1.30it/s]


576870


In [27]:
for i in trainloader:
    print(i)
    break

[tensor([[  101, 14856,  6553,  ...,  1999,  3433,   102],
        [  101,  1996,  2343,  ...,     0,     0,     0],
        [  101,  1996,  2688,  ...,     0,     0,     0],
        ...,
        [  101,  2041,   103,  ...,     0,     0,     0],
        [  101,  2011,   103,  ...,     0,     0,     0],
        [  101,  1999,  3851,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[ 2037,  1996,  2013,  1007,  1997,  2256,  9256,  2028,  2000,  2425],
        [ 2229,  3968,  2000,  2562,  2593,  4259,  2283,  4177,  8573,  2002],
        [ 1012,  2043,  2310, 10009,  

In [6]:
import torch
import torch.distributed as dist
from pathlib import Path
import os
from joblib import Parallel, delayed
import h5py
from tqdm import tqdm
from torch.utils.data import TensorDataset
import numpy as np
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler

class InputFeatures(object):
    """A single set of features of data."""
    def __init__(self, input_ids, input_mask, segment_ids, label_id,next_sentence_labels):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.next_sentence_labels = next_sentence_labels
def _prepare_training_data_helper(type_,train_ids,path,dense_output = False):
    training_samples = []
    for id in tqdm(train_ids):
        path1 = os.path.join(path,type_+'_'+str(id)+'.hdf5')
        f = h5py.File(path1, 'r')
        length = len(f['input_ids'])
        print(length)
        masked_lm_positions = np.array(f['masked_lm_positions'])
        masked_lm_ids = np.array(f['masked_lm_ids'])
        input_ids = np.array(f['input_ids'])
        input_mask = np.array(f['input_mask'])
        segment_ids = np.array(f['segment_ids'])
        next_sentence_labels = np.array(f['next_sentence_labels'])
        max_seq_length = input_mask.shape[1]
        if dense_output:
            for data in range(length):
                lm_label_array = np.full(max_seq_length, dtype=int, fill_value=-1)
                lm_label_array[masked_lm_positions[data]] = masked_lm_ids[data]
                feature = InputFeatures(input_ids=input_ids[data],
                                    input_mask=input_mask[data],
                                    segment_ids=segment_ids[data],
                                    label_id=lm_label_array,
                                    next_sentence_labels =next_sentence_labels[data] )
                training_samples.append(feature)
        else:
            for data in range(length):
                feature = InputFeatures(input_ids=input_ids[data],
                                    input_mask=input_mask[data],
                                    segment_ids=segment_ids[data],
                                    label_id=masked_lm_ids[data],
                                    next_sentence_labels =next_sentence_labels[data])
            # feature = {
            #     'input_ids' : f['input_ids'][data],
            #     'input_mask':f['input_mask'][data],
            #     'segment_ids':f['segment_ids'][data],
            #     'label_id':lm_label_array
            # }
                training_samples.append(feature)
    return training_samples


In [3]:
en_30528_hdf5 = 
a = _prepare_training_data_helper('training',[1,2],en_30528_hdf5,dense_output = False)

NameError: name 'en_30528_hdf5' is not defined

In [4]:
import dllogger
dllogger.init(backends=[dllogger.JSONStreamBackend(verbosity=dllogger.Verbosity.VERBOSE,
                                                           filename=args.json_summary),
                dllogger.StdOutBackend(verbosity=dllogger.Verbosity.VERBOSE, step_format=format_step)])

ModuleNotFoundError: No module named 'dllogger'

In [46]:
100%1000

100

In [11]:
import os
len(os.listdir('output'))

26

In [12]:
26 *6500


169000